# Question 1

Basically, standard deviation says about how much the data is spread out, and standard error of the mean says how much the mean of the dataset is trustworthy, the less - the better

So, while the standard deviation of a dataset is about how much the data varies, the standart error of mean is a good prediction of how much the mean of the dataset would vary if you were to take new other datapoints or add datapoints to the dataset.

# Question 2

To create an n% confidence interval we need to first calculate the SEM, then find the critical value for the required confidence level from the critical value table. In particular, the critical value for 95% is 1.96, so we calculate the confidence interval by the formula (mean +- 1.96 * SEM)

# Question 3

Alternatively to calculating SEM we can just take the 97.5th and 2.5th percentiles of bootstrapped means instead of making calculations; matter fact, ChatGPT says it works better for non-normal distributed data samples

# Question 4

In [6]:
import numpy as np

# Step 1: Generate a random normally distributed sample
np.random.seed(42)  # For reproducibility
sample = np.random.normal(loc=50, scale=10, size=100)  # Mean=50, SD=10, Sample size=100

# Step 2: Define the bootstrap function
def bootstrap_ci(data, statistic_func, n_bootstrap=1000, ci_percentile=95):
    bootstrapped_stats = []
    
    # Generate bootstrap samples and calculate the statistic for each sample
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)  # Sample with replacement
        stat = statistic_func(bootstrap_sample) # in our case it's the mean
        bootstrapped_stats.append(stat)
    
    # Sort bootstrapped statistics
    bootstrapped_stats = np.sort(bootstrapped_stats)
    
    # Calculate the percentiles for the confidence interval
    lower_bound = np.percentile(bootstrapped_stats, (100 - ci_percentile) / 2)
    upper_bound = np.percentile(bootstrapped_stats, 100 - (100 - ci_percentile) / 2)
    
    return lower_bound, upper_bound

# Step 3: Calculate the 95% confidence interval for the mean
ci_mean = bootstrap_ci(sample, np.mean, n_bootstrap=1000, ci_percentile=95)
print(f"95% Bootstrap Confidence Interval for the Mean: {ci_mean}")

# this function is really good because we can use it not only for mean but for basically any other statistic very easily

95% Bootstrap Confidence Interval for the Mean: (47.276635751407504, 50.62851486440195)


# Question 5

We use the confidence interval to say the range of sample statistics that are likely to contain the true population parameter

# Question 6

1. Bootstrapping is when you take a sample of size n, then create a bunch of other samples also of size n only using the elements from the original sample. Duplicates are allowed.
2. To see how much the statistic values might vary if we added more sample data
3. I would bootstrap the sample data a bunch of times (between 1.000 and 10.000 times, depending on how big the provided sample is) so that I see how much the statistics from my sample could be close to the true values by calculating how big are confidence intervals. For example, if my statistic is in the 95% confidence interval and I don't consider the size of the interval to be big then I would 